In [1]:
!pip3 install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 9.8 MB/s eta 0:00:00:00:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached scipy-1.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.8 MB)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder , OneHotEncoder


In [3]:
saved_data = torch.load('../faceEmbeddingModels/embeddings.pt')

In [36]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import cv2


class FaceDetect:

    def __init__(self):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print('Running on device: {}'.format(self.device))
        self.mtcnn = MTCNN(image_size=112, margin=0, device=self.device)
        self.resnet = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)
        saved_data = torch.load('../faceEmbeddingModels/embeddings.pt')
        self.embeddings = saved_data['embeddings']
        self.names = saved_data['names']

    def getFaceEmbedding(self, img):
        face, prob = self.mtcnn(img, return_prob=True)
        if face is not None and prob > 0.70:
            embedding = self.resnet(face.unsqueeze(0).to(self.device)).detach()
            return embedding
        else:
            return None
    def get_prediction(self, embedding):
        dist_list = []
        for idx, emb_db in enumerate(self.embeddings):
            dist = torch.dist(embedding, emb_db).item()
            dist_list.append(dist)
        idx_min = dist_list.index(min(dist_list))
        result_name = self.names[idx_min]
        return result_name
    def detect_face(self):
        cv2.namedWindow("Predicted Name")
        vc = cv2.VideoCapture(0)

        while True:
            rval, frame = vc.read()
            if frame is not None:
                embedding = self.getFaceEmbedding(frame)
                if embedding is not None:
                    result_name = self.get_prediction(embedding)
                    cv2.putText(frame, result_name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("preview", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        cv2.destroyWindow("preview")
        vc.release()


In [ ]:
FaceDetect().detect_face()